In [2]:
import torch
from torch import nn
from pathlib import Path
import random

# from tokenizers import Tokenizer
from torch.utils.data import Dataset, DataLoader
from liger_kernel.transformers import LigerLayerNorm
from liger_kernel.transformers import LigerFusedLinearCrossEntropyLoss

In [3]:

import re
HF_TOKEN = '...'

from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai-community/gpt2", token=HF_TOKEN)
tokenizer.add_special_tokens({"pad_token": "[PAD]"})

SOT = '<|startoftranscript|>'
EOT = '<|endoftranscript|>'
transcribe = '<|transcribe|>'
prev = '<|prev|>'

special_tokens_dict = {
    'additional_special_tokens': [SOT, EOT, transcribe, prev]
}


tokenizer.add_special_tokens(special_tokens_dict)
# model = AutoModelForCausalLM.from_pretrained("openai-community/gpt2")

# tokenizer("hi")

4

In [3]:
import wandb
!wandb login

wandb: Currently logged in as: rajceo2031 (rentio) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [4]:

# !pip install wandb
# import wandb
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("API_KEY")

# wandb.login(key=secret_value_0)

In [1]:
#Hyperparameters
epochs=10
block_size = 64
batch_size = 64
# src_vocab_size = None
tgt_vocab_size = len(tokenizer)
embeddings_dims = 512
attn_dropout = 0.1
no_of_heads = 4 #IMP needs to be thoroughly calculated
dropout = 0.1
# epochs = 3
max_lr = 6e-4
no_of_decoder_layers = 6 #IMP needs to be thoroughly calculated
attn_dropout = 0.1
weight_decay_optim = 0.01
log_mel_features = 80
kernel_size = 3
stride = (2,10)
sr = 16000
device= 'cuda:0'
SAMPLING_RATE=16000
N_MELS = 80  # 80-channel Mel spectrogram
WINDOW_DURATION = 0.025  # 25 milliseconds
STRIDE_DURATION = 0.010  # 10 milliseconds
max_t = 500
n_channels = N_MELS
clip = 1.0
use_flash_attention = False
use_liger = False
use_torch_compile = False

NameError: name 'tokenizer' is not defined

In [6]:
torch.set_default_device(device)

In [7]:

!pip install datasets
from tabnanny import verbose
from datasets import load_dataset

gs = load_dataset("speechcolab/gigaspeech", "xs", token=HF_TOKEN, trust_remote_code=True)


print(gs)


audio_input = gs['train'][0]["audio"]
transcription = gs["train"][0]["text"]

DatasetDict({
    train: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 9389
    })
    validation: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 6750
    })
    test: Dataset({
        features: ['segment_id', 'speaker', 'text', 'audio', 'begin_time', 'end_time', 'audio_id', 'title', 'url', 'source', 'category', 'original_full_path'],
        num_rows: 25619
    })
})


In [8]:
MAX_DURATION_IN_SECONDS = 10

import librosa
from tqdm import tqdm
def is_audio_length_in_range(input_length):
    return input_length < MAX_DURATION_IN_SECONDS

train_new_column = []

for x in tqdm(range(len(gs['test']))):
    train_new_column.append(librosa.get_duration(path=gs['test'][x]['audio']['path']))

gs_ = gs['test'].add_column("duration", train_new_column)


gs_ = gs_.filter(is_audio_length_in_range, input_columns=["duration"])


truncated_gs_train = gs_.remove_columns(["duration"])
# truncated_gs



val_new_column = []
# new_column = [librosa.get_duration(path=x) ]]
for x in tqdm(range(len(gs['validation']))):
    val_new_column.append(librosa.get_duration(path=gs['validation'][x]['audio']['path']))

gs_ = gs['validation'].add_column("duration", val_new_column)


gs_ = gs_.filter(is_audio_length_in_range, input_columns=["duration"])


truncated_gs_val = gs_.remove_columns(["duration"])
# truncated_gs

100%|██████████| 6750/6750 [00:11<00:00, 577.92it/s]


In [9]:

import numpy as np


n_fft = int(WINDOW_DURATION * MAX_DURATION_IN_SECONDS * SAMPLING_RATE)
hop_length = int(STRIDE_DURATION * MAX_DURATION_IN_SECONDS * SAMPLING_RATE)

train_outputs = []
train_texts = []
for i in tqdm(range(len(truncated_gs_train))):
  S = librosa.feature.melspectrogram(
      y=truncated_gs_train[i]['audio']['array'],
      sr=SAMPLING_RATE,
      n_mels=N_MELS,
      n_fft=n_fft,
      hop_length=hop_length,
      win_length=n_fft,
      fmax=SAMPLING_RATE // 2
  )


  S_dB = librosa.power_to_db(S, ref=np.max)
  train_outputs.append(S_dB)
  train_texts.append(truncated_gs_train[i]['text'])

val_outputs = []
val_texts = []
for i in tqdm(range(len(truncated_gs_val))):
  S = librosa.feature.melspectrogram(
      y=truncated_gs_val[i]['audio']['array'],
      sr=SAMPLING_RATE,
      n_mels=N_MELS,
      n_fft=n_fft,
      hop_length=hop_length,
      win_length=n_fft,
      fmax=SAMPLING_RATE // 2
  )


  S_dB = librosa.power_to_db(S, ref=np.max)
  val_outputs.append(S_dB)
  val_texts.append(truncated_gs_val[i]['text'])

  0%|          | 37/22015 [00:00<07:31, 48.72it/s]/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=4000 is too large for input signal of length=1024
  warnings.warn(
  0%|          | 87/22015 [00:01<06:40, 54.74it/s]/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=4000 is too large for input signal of length=3840
  warnings.warn(
  2%|▏         | 352/22015 [00:08<08:15, 43.68it/s]/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=4000 is too large for input signal of length=3056
  warnings.warn(
  2%|▏         | 387/22015 [00:09<08:54, 40.44it/s]/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=4000 is too large for input signal of length=992
  warnings.warn(
  2%|▏         | 399/22015 [00:09<07:30, 47.97it/s]/usr/local/lib/python3.11/dist-packages/librosa/core/spectrum.py:266: UserWarning: n_fft=4000 is too large for input signal 

In [10]:
# import math
# print(round(random.random(), 1))
class GigaSpeechDataset(Dataset):

  def __init__(self, outputs, texts):

    self.data = outputs
    self.texts = texts
    self.max_t = block_size

  def __len__(self):
    return len(self.data)


  def pad_to_max_t(self, spectrogram, max_t):

    n_mels, t = spectrogram.shape
    if t < max_t:
        # Pad with zeros
        pad_width = ((0, 0), (0, max_t - t))
        spectrogram = np.pad(spectrogram, pad_width, mode='constant')
    else:
      spectrogram = spectrogram[:, :max_t]

    return spectrogram

  def clean(self, desc):
    # Use regex to remove anything between < and >
    cleaned_text = re.sub(r'<[^>]*>', '', desc)
    return cleaned_text

  def __getitem__(self, idx):

      SOT = '<|startoftranscript|>'
      EOT = '<|endoftranscript|>'
      transcribe = '<|transcribe|>'
      # prev = '<|prev|>'
      spectrogram = self.pad_to_max_t(self.data[idx], self.max_t)
      # probs = round(random.random(),1)
      spectrogram = torch.tensor(spectrogram, dtype=torch.float32)

      # if(probs == 0.5):
        # Normalize the spectrogram between -1 and 1
      spectrogram_min = spectrogram.min()
      spectrogram_max = spectrogram.max()
      # spectrogram = spectrogram.unsqueeze(0)  # Shape: (1, n_mels, max_t)
      # prev_text =
      text = self.clean(self.texts[idx])

      text = text.lower()
      text = SOT  + 'en' + transcribe +  text + EOT
      tokenized_text = tokenizer(text, truncation=True, padding='max_length', max_length=block_size, return_tensors='pt')
      # print(tokenized_text.shape)

      epsilon = 1e-8  # To avoid division by zero
      spectrogram = 2 * ((spectrogram - spectrogram_min) / (spectrogram_max - spectrogram_min + epsilon)) - 1

      # tokenized_win_prompt = tokenizer(text, max_length = ModelArgs.block_size, padding='max_length', truncation=True,  return_tensors="pt").to(device)
      tokenized_text['labels'] = tokenized_text['input_ids'].clone()
      tokenized_text['labels'][: , :-1] = tokenized_text['input_ids'][: , 1:]
      tokenized_text['labels'][: , -1] = tokenizer.eos_token_id

      tokenized_text_x = tokenized_text['input_ids'].squeeze(0)
      tokenized_text_y = tokenized_text['labels'].squeeze(0)

      # print(tokenized_text.shape)
      return spectrogram, tokenized_text_x, tokenized_text_y

In [11]:
torch.autograd.set_detect_anomaly(True)  # Add at the start of training

In [12]:


shuffle = True

train_dataset = GigaSpeechDataset(train_outputs, train_texts)
val_dataset = GigaSpeechDataset(val_outputs, val_texts)

generator = torch.Generator(device=device)

train_dataloader = DataLoader(

    train_dataset,
    batch_size=batch_size,
    generator=generator,
    shuffle=shuffle,
     drop_last=True,
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    generator=generator,
    drop_last=True ,
    shuffle=shuffle,
)

In [13]:
next(iter(train_dataloader))

[tensor([[[ 0.0780,  0.0936,  0.0573,  ...,  1.0000,  1.0000,  1.0000],
          [-0.0644, -0.0214, -0.0714,  ...,  1.0000,  1.0000,  1.0000],
          [-0.0306,  0.0863,  0.0157,  ...,  1.0000,  1.0000,  1.0000],
          ...,
          [-0.7941, -0.7736, -0.4947,  ...,  1.0000,  1.0000,  1.0000],
          [-0.8894, -0.8927, -0.5498,  ...,  1.0000,  1.0000,  1.0000],
          [-1.0000, -1.0000, -0.7846,  ...,  1.0000,  1.0000,  1.0000]],
 
         [[ 0.1950,  0.3272,  0.2966,  ...,  0.1410,  0.1800,  0.1949],
          [ 0.0017,  0.1503,  0.2539,  ...,  0.6934,  0.6541,  0.3316],
          [ 0.1217,  0.1364,  0.1832,  ...,  0.7802,  0.8221,  0.8245],
          ...,
          [-0.8344, -0.3634,  0.1816,  ..., -0.5553, -0.7330, -0.5329],
          [-0.8927, -0.4419,  0.1594,  ..., -0.6708, -0.8179, -0.6232],
          [-0.9280, -0.5527, -0.0112,  ..., -0.9269, -0.9857, -0.8396]],
 
         [[-0.3366, -0.1151, -0.1663,  ...,  1.0000,  1.0000,  1.0000],
          [-0.1328,  0.1485,

In [14]:
#Position embeddings
class PositionEmbeddings(nn.Module):
    def __init__(
        self,
        embeddings_dims = embeddings_dims,
        block_size = block_size
    ):
        super().__init__()

        self.position_embeddings = nn.Parameter(torch.randn(1, block_size, embeddings_dims, device=device), requires_grad=True) #To give positional embeddings to each token of the input text, hence num_embeddings=block_size
        # nn.init.normal_(self.position_embeddings.weight.data, mean=0, std=0.02)

    def forward(self, x):
        return self.position_embeddings

In [15]:
pos = PositionEmbeddings()
x = torch.randn(batch_size, block_size, embeddings_dims)
pos(x)

Parameter containing:
tensor([[[ 0.3495,  0.6073, -1.3787,  ..., -0.6898,  2.2222, -1.2482],
         [ 1.2467,  0.5727, -0.5820,  ...,  0.7372, -0.1024,  1.8049],
         [ 1.9835,  1.1466, -1.0870,  ...,  0.3833,  1.0924, -1.0911],
         ...,
         [-0.4632, -0.1689, -0.2957,  ...,  0.1328, -0.2095, -1.5386],
         [ 0.2848, -1.3799,  1.5286,  ..., -0.3578, -0.1225, -0.7640],
         [ 1.0754,  1.2897, -1.1724,  ...,  0.4539, -0.2270, -1.5164]]],
       device='cuda:0', requires_grad=True)

In [16]:


# Text embeddings
class TgtTextEmbeddings(nn.Module):
    def __init__(
        self,
        vocab_size = tgt_vocab_size,
        embeddings_dims = embeddings_dims
    ):
        super().__init__()
        self.embeddings_table = nn.Embedding(num_embeddings = tgt_vocab_size, embedding_dim=embeddings_dims, device=device) #Just a look up table to convert the toekns_ids to some numbers
        # nn.init.normal_(self.embeddings_table.weight.data, mean=0, std=0.02)

    def forward(self, x):
        return self.embeddings_table(x)

In [17]:



#Layer Normalization

class LayerNormalization(nn.Module):
    def __init__(
        self,
        embeddings_dims = embeddings_dims
    ):
        super().__init__()
        if(use_liger == False):
            self.norm = nn.LayerNorm(normalized_shape=embeddings_dims)
        else:
            self.norm = LigerLayerNorm(normalized_shape=embeddings_dims)

    def forward(self, x):

        return self.norm(x)

In [18]:



#FeedForward Neural Network

class MLPBlock(nn.Module):
    def __init__(
        self,
        dropout = dropout,
        embeddings_size = embeddings_dims,
        # inner_dimensional_states: int = 3072
    ):
        super().__init__()

        self.mlp = nn.Sequential(
            nn.Linear(device=device, in_features=embeddings_size, out_features= 4 * embeddings_dims),
            nn.GELU(),
            nn.Linear(device=device, in_features= 4 * embeddings_dims, out_features=embeddings_size),
            nn.Dropout(p = dropout)
        )

    def forward(self, x):
        # mlp_weights_init = self.mlp.apply(weights_init)
        return self.mlp(x)

In [19]:


class MaskedAttentionHead(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
    ):
        super().__init__()
        self.head_size = embeddings_dims // no_of_heads
        self.query = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=device, bias=False)
        self.keys = nn.Linear(in_features=embeddings_dims, out_features=self.head_size,device=device, bias=False)
        self.values = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=device,bias=False)
        self.dropout = nn.Dropout(p = attn_dropout)


    def forward(self, x):
        # print(x.shape)
        batch, block_size, embd_dims = x.shape
        k = self.keys(x)
        q = self.query(x)
        v = self.values(x)
        if(use_flash_attebntion == False):
            masked_table = torch.tril(torch.ones(block_size, block_size, device=device))
            weights = q @ torch.transpose(k, dim0=-2, dim1=-1) * (k.shape[-1] ** -0.5)
            masked_values = weights.masked_fill(masked_table[: block_size, : block_size] == 0, float('-inf'))
            weights_normalized = nn.functional.softmax(masked_values, dim=-1) #Normalize along the embeddings dimension for all the tokens
            weights_normalized = self.dropout(weights_normalized)
            out = weights_normalized @ v
            return out
        else:
            out = torch.nn.functional.scaled_dot_product_attention(q, k, v, dropout_p=dropout ,is_causal=True)
            return out

In [20]:



class MaskedMHA(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
    ):
        super().__init__()
        self.heads = nn.ModuleList([MaskedAttentionHead(attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads) for _ in range(no_of_heads)])
        self.dropout = nn.Dropout(p = attn_dropout)
        self.linear = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=device, bias=False) # 12 (no of heads) * (batch_size) 64 = 768 -> gives out the text embeddings

    def forward(self, x):
        concat = torch.cat([head(x) for head in self.heads], dim=-1)
        linear_layer = self.linear(concat)
        out = self.dropout(linear_layer)
        return out

In [21]:

#Single Attention Head

class CrossAttentionHead(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
    ):
        super().__init__()
        self.head_size = embeddings_dims // no_of_heads
        self.query = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=device, bias=False)
        self.keys = nn.Linear(in_features=embeddings_dims, out_features=self.head_size,device=device, bias=False)
        self.values = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=device,bias=False)
        self.dropout = nn.Dropout(p = attn_dropout)


    def forward(self, query, key, value, mask=None):


        batch, block_size, embd_dims = query.shape
        q = self.query(query)
        k = self.keys(key)
        v = self.values(value)
        # masked_table = torch.tril(torch.ones(block_size, block_size, device=device))
        # weights = query @ torch.transpose(key, dim0=-2, dim1=-1) * (key.shape[-1] ** -0.5)
        # if(mask != None):
        #     mask = mask.unsqueeze(1)
        #     masked_values = weights.masked_fill(mask == 0, float('-inf'))
        #     weights_normalized = nn.functional.softmax(masked_values, dim=-1) #Normalize along the embeddings dimension for all the tokens
        #     # weights_normalized = self.dropout(weights_normalized)
        #     out = weights_normalized @ value
        #     out = self.dropout(out)
        #     return out
        # else:
        #     weights_normalized = nn.functional.softmax(weights, dim=-1) #Normalize along the embeddings dimension for all the tokens
        #     # weights_normalized = self.dropout(weights_normalized)
        #     out = weights_normalized @ value
        #     out = self.dropout(out)
        #     return out

        masked_table = torch.tril(torch.ones(block_size, block_size, device=device))
        weights = q @ torch.transpose(k, dim0=-2, dim1=-1) * (k.shape[-1] ** -0.5)
        masked_values = weights.masked_fill(masked_table[: block_size, : block_size] == 0, float('-inf'))
        weights_normalized = nn.functional.softmax(masked_values, dim=-1) #Normalize along the embeddings dimension for all the tokens
        weights_normalized = self.dropout(weights_normalized)
        out = weights_normalized @ v
        return out

In [22]:
#Single Attention Head

class FullAttentionHead(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
    ):
        super().__init__()
        self.head_size = embeddings_dims // no_of_heads
        self.query = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=device, bias=False)
        self.keys = nn.Linear(in_features=embeddings_dims, out_features=self.head_size,device=device, bias=False)
        self.values = nn.Linear(in_features=embeddings_dims, out_features=self.head_size, device=device,bias=False)
        self.dropout = nn.Dropout(p = attn_dropout)


    def forward(self, x, mask=None):
        # batch, block_size, embd_dims = x.shape
        k = self.keys(x)
        q = self.query(x)
        v = self.values(x)
        # masked_table = torch.tril(torch.ones(block_size, block_size, device=device))
        weights = q @ torch.transpose(k, dim0=-2, dim1=-1) * (k.shape[-1] ** -0.5)
        if(mask != None):
            mask = mask.unsqueeze(1)
            masked_values = weights.masked_fill(mask == 0, float('-inf'))
            weights_normalized = nn.functional.softmax(masked_values, dim=-1) #Normalize along the embeddings dimension for all the tokens
            # weights_normalized = self.dropout(weights_normalized)
            out = weights_normalized @ v
            out = self.dropout(out)
            return out
        else:
            weights_normalized = nn.functional.softmax(weights, dim=-1) #Normalize along the embeddings dimension for all the tokens
            # weights_normalized = self.dropout(weights_normalized)
            out = weights_normalized @ v
            out = self.dropout(out)
            return out

In [23]:

class FullMHA(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
    ):
        super().__init__()
        self.heads = nn.ModuleList([FullAttentionHead(attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads) for _ in range(no_of_heads)])
        self.dropout = nn.Dropout(p = attn_dropout)
        self.linear = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=device, bias=False) # 12 (no of heads) * (batch_size) 64 = 768 -> gives out the text embeddings

    def forward(self, x, mask=None):
        concat = torch.cat([head(x, mask) for head in self.heads], dim=-1)
        linear_layer = self.linear(concat)
        out = self.dropout(linear_layer)
        return out

In [24]:


class CrossMHA(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
    ):
        super().__init__()
        self.heads = nn.ModuleList([CrossAttentionHead(attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads) for _ in range(no_of_heads)])
        self.dropout = nn.Dropout(p = attn_dropout)
        self.linear = nn.Linear(in_features=embeddings_dims, out_features=embeddings_dims, device=device, bias=False)

    def forward(self, value, key, x, mask=None):
        concat = torch.cat([head(x, key, value,  mask) for head in self.heads], dim=-1)
        linear_layer = self.linear(concat)
        out = self.dropout(linear_layer)
        return out

In [25]:
# Decoder Block

class TransformerDecoderBlock(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
        dropout = dropout,
        # vocab_size = vocab_size
    ):
        super().__init__()

        self.cross = CrossMHA(attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads)
        self.masked = MaskedMHA(attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads)
        self.layer_norm1 = LayerNormalization(embeddings_dims)
        self.layer_norm2 = LayerNormalization(embeddings_dims)
        # self.layer_norm3 = LayerNormalization(embeddings_dims=embeddings_dims)
        self.layer_norm4 = LayerNormalization(embeddings_dims)
        self.mlp_block = MLPBlock(dropout=dropout, embeddings_size=embeddings_dims)

    def forward(self, key, value, x, mask=None):
        x = self.layer_norm1(x + self.masked(x)) #Very important step -> Layer Norm on input and then passes it to the subsequent blocks
        # print(x.shape)
        x = self.layer_norm2(x + self.cross(value, key, x, mask)) #Very important step
        # print(x.shape)
        # x = x + self.mha(self.layer_norm1(x))  #Very important step -> Layer Norm on input and then passes it to the subsequent blocks
        x = self.layer_norm4(x + self.mlp_block(x)) #Very important step
        # print(x.shape)

        return x

In [26]:
# Decoder Block

class DecoderModel(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
        block_size = block_size,
        dropout = dropout,
        no_of_decoder_layers = no_of_decoder_layers,
        # vocab_size = vocab_size
    ):
        super().__init__()




        # self.tgt_text_embds = TgtTextEmbeddings(vocab_size=tgt_vocab_size, embeddings_dims=embeddings_dims)
        # self.linear_layer = nn.Linear(in_features=embeddings_dims, out_features=tgt_vocab_size, device=device, bias=False) # Takes in logits of dimensions- embeds_dims and converts it into dimension of vocab_size (logits in range of vocab_size)
        # self.layer_norm = LayerNormalization(embeddings_dims=embeddings_dims)
        self.decoder_layers = nn.ModuleList([TransformerDecoderBlock(attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads, dropout=dropout) for _ in range(no_of_decoder_layers)])
        self.apply(self._init_weights)
        # self.positional_embeddings_tgt = nn.Parameter(torch.randn(1, block_size, embeddings_dims, device=device), requires_grad=True) #To give positional embeddings to each token of the input text, hence num_embeddings=block_size
        self.positional_embeddings_tgt = PositionEmbeddings()
        # torch.nn.init.normal_(self.positional_embeddings_tgt, mean=0.0, std=0.02)

        # out = self.decoder_layers(query, key, x)
        # Loop through each decoder layer
    def _init_weights(self, module):  #Weight Initialization
            if isinstance(module, nn.Linear):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
                if module.bias is not None:
                    torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, key, value, x, mask):
        # x = self.tgt_text_embds(x)
        x = x + self.positional_embeddings_tgt(x)
        # print(x.shape)
        for decoder_layer in self.decoder_layers:
            x = decoder_layer(key, value, x, mask)
        # x = self.layer_norm(x)

        return x

In [27]:

#Encoder

In [28]:



class TransformerEncoderBlock(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
        dropout = dropout,
        mask=None
    ):
        super().__init__()

        self.mha = FullMHA(attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads)
        self.layer_norm1 = LayerNormalization(embeddings_dims)
        self.layer_norm2 = LayerNormalization(embeddings_dims)
        self.mlp_block = MLPBlock(dropout=dropout, embeddings_size=embeddings_dims)

    def forward(self, x, mask=None):
        x = self.layer_norm1(x + self.mha(x, mask))
        x = self.layer_norm2(x + self.mlp_block(x))

        return x

In [29]:



class EncoderModel(nn.Module):
    def __init__(
        self,
        attn_dropout = attn_dropout,
        embeddings_dims = embeddings_dims,
        no_of_heads = no_of_heads,
        block_size = block_size,
        dropout = dropout,
        no_of_decoder_layers = no_of_decoder_layers,
        # vocab_size = vocab_size
    ):
        super().__init__()


        # self.positional_embeddings_src = nn.Parameter(torch.randn(1, block_size, embeddings_dims, device=device), requires_grad=True) #To give positional embeddings to each token of the input text, hence num_embeddings=block_size

        self.conv1 = nn.Conv1d(in_channels=n_channels, out_channels=embeddings_dims, kernel_size=kernel_size, device=device, padding=1)
        self.conv2 = nn.Conv1d(in_channels=embeddings_dims, out_channels=embeddings_dims, kernel_size=kernel_size, device=device, padding=1)

        self.positional_embeddings_src = PositionEmbeddings()

        self.encoder_layers = nn.ModuleList([TransformerEncoderBlock(attn_dropout=attn_dropout, embeddings_dims=embeddings_dims, no_of_heads=no_of_heads, dropout=dropout) for _ in range(no_of_decoder_layers)])
        self.apply(self._init_weights)

    def _init_weights(self, module):  #Weight Initialization
            if isinstance(module, nn.Linear):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
                if module.bias is not None:
                    torch.nn.init.zeros_(module.bias)
            elif isinstance(module, nn.Embedding):
                torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, x, mask):

        x = self.conv1(x)
        x = torch.nn.functional.gelu(x)
        x = self.conv2(x)
        x = torch.nn.functional.gelu(x)
        # print(x.shape)
        # x = self.src_text_embeds(x)
        # print(self.positional_embeddings_src.shape)
        x = x.permute(0, 2, 1)
        # print(x.shape)
        # print(self.positional_embeddings_src(x).shape)
        x = x + self.positional_embeddings_src(x)
        # print(x)
        # print(x.shape)
        # Loop through each encoder layer
        for encoder_layer in self.encoder_layers:
            x = encoder_layer(x, mask)
        return x

In [30]:


class Transformer(nn.Module):
    def __init__(
        self,

    ):
        super().__init__()

        self.encoder = EncoderModel()
        self.decoder = DecoderModel()
        # self.pos = PositionalEmbeddings()
        self.tgt_text_embds = TgtTextEmbeddings(vocab_size=tgt_vocab_size, embeddings_dims=embeddings_dims)
        self.linear_layer = nn.Linear(in_features=embeddings_dims, out_features=tgt_vocab_size, device=device, bias=False) # Takes in logits of dimensions- embeds_dims and converts it into dimension of vocab_size (logits in range of vocab_size)
        # self.src_text_embeds = SrcTextEmbeddings(vocab_size=src_vocab_size, embeddings_dims=embeddings_dims)

    def forward(self, src, tgt, src_mask=None, tgt_mask=None, actual_labels=None):
        # x = self.src_text_embeds(src)
        x = self.encoder(src, src_mask)
        y = self.tgt_text_embds(tgt)
        # print(x.shape)
        y = self.decoder(x, x, y, tgt_mask)
        # print(y.shape)
        if(use_liger):
            linear = self.linear_layer(y)
            res = LigerFusedLinearCrossEntropyLoss(linear.weight, actual_labels, y)
            return res
        else:
            out = self.linear_layer(y)
        return out

In [31]:
#Instantiating the model
model = Transformer()
# model = torch.compile(model)
# model = model.to(device)
model = model.to(device)

In [32]:
import gc
gc.collect()

1220

In [33]:
tgt_mask = torch.randint(1, tgt_vocab_size, (batch_size, block_size)).to(device)  #
tgt_mask

tensor([[40881, 40758, 39215,  ...,  4273, 33545, 12089],
        [41156, 12515,  1242,  ..., 44457, 34477, 41097],
        [13258, 45885, 48303,  ..., 42415,  7490, 39716],
        ...,
        [14931, 15291, 26114,  ..., 29419,  8344, 41207],
        [26139, 37021, 31918,  ..., 32901, 26692, 38368],
        [47027, 32866,  2893,  ...,  1025, 11696, 12860]], device='cuda:0')

In [34]:

!pip install torchinfo
from torchinfo import summary

spec, text = next(iter(train_dataloader))
tgt_mask = torch.randint(1, tgt_vocab_size, (batch_size, block_size)).to(device)  #
spec = spec.to(device)
texts = text.to(device)

summary(model=model,
        input_data=(spec, texts),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

In [35]:

# # Optimizer setup and scheduler steup
# out = {"Train": None, "val": None}
optimizer = torch.optim.AdamW(model.parameters(), lr=max_lr)

loss_fn = nn.CrossEntropyLoss()

In [36]:
torch.set_float32_matmul_precision('high')

scaler = torch.amp.GradScaler(enabled=True)

In [37]:
def _save_snapshot(model, optimizer, scheduler, epoch, step):
    snapshot = {
        "MODEL_STATE": model.state_dict(),
        "OPTIMIZER_STATE": optimizer.state_dict(),
        # "SCHEDULER_STATE": scheduler.state_dict(),
        "EPOCHS_RUN": epoch,
        "STEP_RUN": step
    }
    torch.save(snapshot, f"snapshot_{step}.pt")
    print(f"Epoch: {epoch} | Step: {step} | Snapshot saved.")

In [38]:
# !pip install torchtriton

In [39]:
save_chechpoint_iter = 50
total_iters = 20000
eval_iters = 50
eval_check = 100
warmup_iters = 2048
min_lr = 0.1 * max_lr
lr_decay_iters = 20000
total_batch_size = 524288
micro_batch_size = batch_size
gradient_accumulation_steps = total_batch_size // (micro_batch_size * (block_size * torch.cuda.device_count()))

In [40]:
model.eval()
world_size = torch.cuda.device_count()
@torch.inference_mode()
def estimate_loss(val_loader, val_iterator, device):
    out = {}
    # train_loader = prepare_dataset('train', ModelArgs.batch_size)

    # val_loader_iterator = iter(val_loader)
    loader = None
    epoch_loss = None
    epoch_losses = []
    # print("Starting the eval...")
    for split in ['val']:
        print(f"Starting with {split} evaluation...")
        # losses = torch.zeros(ModelArgs.val_epochs)
        # if(split == 'train'):
        #         loader = train_loader
        # if(split == 'val'):
        #         loader = val_loader
        for step in range(eval_check):
            try:
                batch = next(val_iterator)
            except StopIteration:
                val_loader_iterator = iter(val_loader)
                X, x, y = next(val_loader_iterator)

            # tgt_mask = torch.randint(1, tgt_vocab_size, (batch_size, block_size)).to(device)  #
            total_loss = 0
            # loader.sampler.set_epoch(step)
            total_batches = 0
            # batch = next(val_loader_iterator)
            # for batch in loader:  # Loop through DataLoader batches
            # idx = batch['input_ids']
            # targets = batch['labels']
            spec = X.to(device)

            idx = x.to(device)
            targets = y.to(device)
            with torch.autocast(device_type=device, dtype=torch.float16):

                logits = model(spec, idx)
                batch_size, block_size, embeddings_dims = logits.shape
                logits = logits.view(batch_size * block_size, embeddings_dims)  # Flatten tokens
                targets = targets.view(batch_size * block_size)

                loss = torch.nn.functional.cross_entropy(logits, targets, ignore_index=tokenizer.pad_token_id)

                total_loss += loss.item()
                total_batches += 1

        # Compute mean loss for this epoch
        epoch_loss = total_loss / total_batches if total_batches > 0 else 0.0
        epoch_losses.append(epoch_loss)

            # print(f"Epoch {epoch + 1}/{ModelArgs.val_epochs}: Loss = {epoch_loss:.4f}")

        # Compute mean loss across all evaluation epochs
        out[split] = sum(epoch_losses) / len(epoch_losses) if epoch_losses else 0.0
        epoch_loss = None
        epoch_losses = []

    model.train()
    return out

# model = model.to(rank)
model.train()
count = 0

# train_dataloader = prepare_dataset('train', device, ModelArgs.batch_size)
# val_loader= prepare_dataset('val', device, ModelArgs.batch_size)
# for step in tqdm(range(total_iters)):
# for epoch in range(ModelArgs.epochs):
    # torch.cuda.synchronize()

# train_dataloader.sampler.set_epoch(epoch)

# val_loader.sampler.set_epoch(epoch)
print("Loaders ready both")
epochs = epochs

# train_step_iterator = range(len(train_dataloader))
# if device == 0:  # Only create progress bar on rank 0
#   train_step_iterator = tqdm(train_step_iterator, desc="Training Progress", position=0, leave=True)

    # Print progress on rank 0
train_loader_length = 0
train_data_iterator = iter(train_dataloader)
val_data_iterator = iter(val_dataloader)
token_count = 0
if(device == 0):
    train_loader_length = len(train_dataloader)
    # print("Total batches: ", train_loader_length)
# print("Length of : ", len(train_dataloader))
# print("Length of val: ", len(val_loader))
# for  step, batch in enumerate(train_dataloader):

Loaders ready both


In [41]:
def find_unused_parameters(model):
    unused = []
    for name, param in model.named_parameters():
        if param.grad is None:
            unused.append(name)
    return unused

In [42]:
import math
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return max_lr * (it + 1) / (warmup_iters + 1)
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)

In [43]:
#

In [ ]:
model.train()
train_losses =  torch.zeros(len(train_dataloader))
val_losses = torch.zeros(len(val_dataloader))
wandb.init(
    project='Whisper-From-Scratch'
)
step = 0
if(use_torch_compile):
    model = torch.compile(model)

# loss_fn = LigerFusedLinearCrossEntropyLoss()
for step in tqdm(range(total_iters)):
        # print("Dataloader things: ", batch)
        # print("Total batches: ", len(train_dataloader))


        # if(device == 0):
            # if(step % 100 == 0):
        #     if(step == train_loader_length):
        #       break
        print("Step : ", step, "/", total_iters)
        print('Total batches: ', len(train_dataloader))
        print("Total gradient accumulation steps: ", gradient_accumulation_steps)
                # print("Total tokens processed: ", token_count)

        # all_gpus_avg_train_loss = None
        # all_gpus_avg_val_loss = None
        # every once in a while evaluate the loss on train and val sets
        if (step  % eval_iters == 0 and step != 0) or step == total_iters - 1:
            losses = estimate_loss( val_loader, val_data_iterator, 'cuda')
            # avg_train_loss = losses['train']
            avg_val_loss = losses['val']
            # print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
            # if device == 0:  # Only print on main process
            print(f"[GPU {device}] | Step: {step} / {total_iters} | Val Loss: {losses['val']:.4f}")
            # print(f"[GPU {device}] | Epoch {epoch}/{ModelArgs.epochs}| |Step: {step} | Train Loss: {losses['train']:.4f}")
                # print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
                # Log training loss more frequently
                # Aggregate average loss across all GPUs
            # avg_train_loss = torch.Tensor([losses['train']]).to(device)
            avg_val_loss = torch.Tensor([losses['val']]).to(device)
            # torch.distributed.reduce(avg_train_loss, dst=0, op=torch.distributed.ReduceOp.SUM)
            # torch.distributed.reduce(avg_val_loss, dst=0, op=torch.distributed.ReduceOp.SUM)

            # if device == 0:
                # all_gpus_avg_train_loss = avg_train_loss / world_size
                # print(f"All_GPUs_Train_losses: {all_gpus_avg_train_loss.item():.4f}")
            all_gpus_avg_val_loss = avg_val_loss / world_size
            print(f"All_GPUs_Val_losses: {all_gpus_avg_val_loss.item():.4f}")

            # if device == 0:

                # writer.add_scalar("All_GPUs_Train_losses", all_gpus_avg_train_loss.item(), global_step=step)
                # writer.add_scalar("All_GPUs_Val_losses", all_gpus_avg_val_loss.item(), global_step=step)
                # writer.add_scalar("training_step_loss", losses['train'], global_step=step)
                # writer.add_scalar("val_step_loss", losses['val'], global_step=step)
                # writer.add_scalar("GPU", device, global_step=step)
                # writer.add_scalar("Epoch", epoch, global_step=step)

            wandb.log({
                    # "Learning Rate": optimizer.param_groups[0]['lr'],
                    # "All_GPUs_Train_losses": all_gpus_avg_train_loss,
                    "All_GPUs_Val_losses": all_gpus_avg_val_loss,
                    # "training_step_loss": losses['train'],
                    "val_step_loss": losses['val'],
                    # "Step": step,
                    # "Epoch": epoch
                })



        #Loading a checkpoint
        # if(os.path.exists('snapshot.pt')):
        #    model, optimizer =  _load_snapshot(model=model, optimizer=optimizer, epoch=epoch, step=step, snapshot_path='snapshot.pt')

        # if(step % save_chechpoint_iter == 0 and device == 0 and step != 0):

        #     _save_snapshot(epoch=epoch, model=model, optimizer=optimizer, step=step)

        if step % save_chechpoint_iter == 0 and device == 0 and step != 0:
            print(f"Saving the model checkpoint for step: {step}")
            _save_snapshot(model, optimizer, None, None, step)

        accumulated_loss = 0.0


        optimizer.zero_grad(set_to_none=True)
        for micro_step in range(gradient_accumulation_steps):
            try:
                spec, idx, y = next(train_data_iterator)
            except StopIteration:
                train_data_iterator = iter(train_dataloader)
                spec, idx, y = next(train_data_iterator)
            spec = spec.to(device)
            y = y.to(device)
            idx = idx.to(device)

            # tgt_mask = torch.randint(1, tgt_vocab_size, (batch_size, block_size)).to(device)  #
            # print(batch)
            # batch = next(train_data_iterator)
            # print(batch)
            # batch = {k: v.to(self.local_rank) for k, v in batch.items()}
            # idx = batch['input_ids'].to(device)
            # idx, targets = get_batch(split='train')
            # print(f"Starting the train step: {step}...")
            # for idx, targets in train_loader:
            # idx, targets = next(iter(train_loader))

            # print("Idx: ", idx)
            # print("Targets: ", targets)

            # idx = idx.to(device)
            # print("Idx: ", idx)
            # print("Targets: ", targets)
            # targets = batch['labels'].to(device)
            # token_count += len(idx)
            with torch.autocast(device_type=device, dtype=torch.float16):
                logits = model(spec, idx)
                batch_size, block_size, embeddings_dims = logits.shape
                # print(logits.shape)
                # print(targets)
                logits = logits.view(batch_size*block_size, embeddings_dims)
                # print("OK")
                targets = y.view(batch_size * block_size)
                # print("OK2")
                if(use_liger == False):
                    loss_fn
                else:
                    loss = nn.functional.cross_entropy(logits, targets, ignore_index=tokenizer.pad_token_id)

                loss = loss / gradient_accumulation_steps #IDK why div is done here specifically? Maybe think of it in terms of a very big batch being processed and there is need for equal important of each mini batch for the overall big batch
                accumulated_loss += loss.detach()

            model.require_backward_grad_sync = (micro_step == gradient_accumulation_steps - 1) # so that we dont synchronize the gradient everytime across the GPU devices
            scaler.scale(loss).backward()
            # print("loss: ", loss.item())
                # Check for unused parameters
            unused_params = find_unused_parameters(model)
            if unused_params:
                print(f"Unused parameters: {unused_params}")
        # break

            # if(device == 0):
            if(micro_step % 10 == 0):
            #     if(step == train_loader_length):
            #       break

                    print("Micro Batch : ", micro_step)
                    print("Step : ", step, "/", total_iters)
                    print('Total batches: ', len(train_dataloader))
                    print("Total gradient accumulation steps: ", gradient_accumulation_steps)
                    print("Total tokens processed: ", token_count)
            # count += 1

        lr = get_lr(step)
        for params in optimizer.param_groups:
            params['lr'] = lr



        # Compute gradient norms before clipping
        if(clip != 0.0):
            scaler.unscale_(optimizer) #To avoid underflow
            total_norm_before = torch.norm(
                torch.stack([torch.norm(p.grad.detach(), 2) for p in model.parameters()]), 2
            )

            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=clip)

            # Compute gradient norms after clipping
            total_norm_after = torch.norm(
                torch.stack([torch.norm(p.grad.detach(), 2) for p in model.parameters()]), 2
            )

            if(device  == 0 and step !=0):
                print(f"Gradient Norm Before Clipping: {total_norm_before.item():.4f}")
                print(f"Gradient Norm After Clipping: {total_norm_after.item():.4f}")

        scaler.step(optimizer)
        scaler.update()

        # optimizer.step()
        # new_scheduler.step()
        print(accumulated_loss)
        # torch.cuda.synchronize()
        # torch.distributed.reduce(loss, dst=0, op=torch.distributed.ReduceOp.SUM)
        # if(device == 0):
        wandb.log({
                    "Learning Rate": lr,
                    "All_GPUs_Train_losses": accumulated_loss.item(),
                    # "All_GPUs_Val_losses": all_gpus_avg_val_loss,
                    # "training_step_loss": losses['train'],
                    # "val_step_loss": losses['val'],
                    "Step": step,
                    # "Epoch": epoch

                })


        # model.train()
        # wandb.log({
        #   "Train Loss": train_losses.mean(),
        #   "Val Loss": val_losses.mean(),
        #   # "epoch": epoch
        # })
        # print("Epoch: ", epoch, "|", "Train Loss: ", train_losses.mean(),  "|", "Val Loss: ", val_losses.mean())

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: rajceo2031 (rentio) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


  0%|          | 0/20000 [00:00<?, ?it/s]

Step :  0 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Micro Batch :  0
Step :  0 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  10
Step :  0 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  20
Step :  0 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  30
Step :  0 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  40
Step :  0 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  50
Step :  0 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  60
Step :  0 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  70
Step :  0 / 20000
Total batches:  343
Total 

  0%|          | 1/20000 [03:33<1184:04:50, 213.15s/it]

tensor(11.0315, device='cuda:0')
Step :  1 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Micro Batch :  0
Step :  1 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  10
Step :  1 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  20
Step :  1 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  30
Step :  1 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  40
Step :  1 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  50
Step :  1 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  60
Step :  1 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  70
Step :  1 /

  0%|          | 2/20000 [07:04<1177:06:20, 211.90s/it]

tensor(11.0287, device='cuda:0')
Step :  2 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Micro Batch :  0
Step :  2 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  10
Step :  2 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  20
Step :  2 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  30
Step :  2 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  40
Step :  2 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  50
Step :  2 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  60
Step :  2 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  70
Step :  2 /

  0%|          | 3/20000 [10:36<1178:52:47, 212.23s/it]

tensor(11.0208, device='cuda:0')
Step :  3 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Micro Batch :  0
Step :  3 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  10
Step :  3 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  20
Step :  3 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  30
Step :  3 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  40
Step :  3 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0
Micro Batch :  50
Step :  3 / 20000
Total batches:  343
Total gradient accumulation steps:  128
Total tokens processed:  0


In [ ]:
device